In [1]:
import pandas as pd
import sasoptpy as so
import os

In [2]:
df = pd.read_excel(r"P:\Projects\FPL\Data\Single GW.xlsx")
df

,Pos_Num,Pos,Name,BV,SV,Team,23_xMins,23_Pts
0,1,G,Lloris,5.6,5.6,Spurs,88,4.152
1,2,D,Alonso,5.6,5.6,Chelsea,77,4.617
2,2,D,Azpilicueta,5.6,5.6,Chelsea,74,4.063
3,2,D,Thiago Silva,5.6,5.6,Chelsea,77,4.029
4,2,D,Dier,5.0,5.0,Spurs,84,4.005
...,...,...,...,...,...,...,...,...
640,4,F,Simms,4.5,4.5,Everton,0,0.000
641,4,F,Hackford,4.4,4.4,Sheffield Utd,0,0.000
642,4,F,Diaby,4.5,4.5,West Brom,0,0.000
643,4,F,Cutrone,5.9,5.9,Wolves,0,0.000


In [3]:
gk_data = df[df['Pos_Num']==1].copy().reset_index()
gk_data

,index,Pos_Num,Pos,Name,BV,SV,Team,23_xMins,23_Pts
0,0,1,G,Lloris,5.6,5.6,Spurs,88,4.152
1,11,1,G,Sánchez,4.4,4.4,Brighton,88,4.030
2,15,1,G,Mendy,5.1,5.1,Chelsea,88,4.229
3,16,1,G,de Gea,5.3,5.3,Man Utd,87,3.826
4,17,1,G,Schmeichel,5.5,5.5,Leicester,87,3.812
...,...,...,...,...,...,...,...,...,...
69,82,1,G,Langley,4.0,4.0,Newcastle,0,0.000
70,83,1,G,Carson,4.0,4.0,Man City,0,0.000
71,84,1,G,Söndergaard,4.0,4.0,Wolves,0,0.000
72,85,1,G,Lewis,4.0,4.0,Southampton,0,0.000


In [4]:
model = so.Model('gk_model')

NOTE: Initialized model gk_model.


In [5]:
gk_data.set_index('index',inplace=True)

In [6]:
gk_data.head()

,Pos_Num,Pos,Name,BV,SV,Team,23_xMins,23_Pts
index,,,,,,,,
0,1,G,Lloris,5.6,5.6,Spurs,88,4.152
11,1,G,Sánchez,4.4,4.4,Brighton,88,4.030
15,1,G,Mendy,5.1,5.1,Chelsea,88,4.229
16,1,G,de Gea,5.3,5.3,Man Utd,87,3.826
17,1,G,Schmeichel,5.5,5.5,Leicester,87,3.812


In [7]:
players = gk_data.index.tolist()

In [8]:
lineup = model.add_variables(players,name='lineup',vartype=so.binary)

In [9]:
lineup

sasoptpy.VariableGroup([0, 11, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86], name='lineup')

In [10]:
bench = model.add_variables(players,name='bench',vartype=so.binary)

In [11]:
lineup[1]

C:\Users\Pankti Prajwolika\AppData\Local\Temp\ipykernel_13412\2853833603.py:1: RuntimeWarning: Requested variable group is empty: lineup[1] (<class 'int'>)
  lineup[1]


In [12]:
total_xp = so.expr_sum(lineup[p]*gk_data.loc[p,'23_Pts'] for p in players) + 0.1* so.expr_sum(bench[p]*gk_data.loc[p,'23_Pts'] for p in players)

In [13]:
total_xp

sasoptpy.Expression(exp = 4.152 * lineup[0] + 4.03 * lineup[11] + 4.229 * lineup[15] + 3.826 * lineup[16] + 3.812 * lineup[17] + 3.737 * lineup[18] + 3.721 * lineup[19] + 3.654 * lineup[20] + 3.643 * lineup[21] + 3.536 * lineup[22] + 3.439 * lineup[23] + 3.428 * lineup[24] + 3.401 * lineup[25] + 3.371 * lineup[26] + 3.183 * lineup[27] + 3.097 * lineup[28] + 3.095 * lineup[29] + 2.488 * lineup[30] + 1.716 * lineup[31] + 1.446 * lineup[32] + 1.384 * lineup[33] + 1.22 * lineup[34] + 0.664 * lineup[35] + 0.08 * lineup[36] + 0.4152 * bench[0] + 0.403 * bench[11] + 0.4229 * bench[15] + 0.3826 * bench[16] + 0.3812 * bench[17] + 0.3737 * bench[18] + 0.3721 * bench[19] + 0.3654 * bench[20] + 0.3643 * bench[21] + 0.3536 * bench[22] + 0.3439 * bench[23] + 0.3428 * bench[24] + 0.3401 * bench[25] + 0.3371 * bench[26] + 0.3183 * bench[27] + 0.3097 * bench[28] + 0.3095 * bench[29] + 0.2488 * bench[30] + 0.1716 * bench[31] + 0.1446 * bench[32] + 0.1384 * bench[33] + 0.122 * bench[34] + 0.0664 * bench[

In [14]:
model.set_objective(total_xp, name='total_xp_obj', sense = so.maximize)

sasoptpy.Expression(exp = 4.152 * lineup[0] + 4.03 * lineup[11] + 4.229 * lineup[15] + 3.826 * lineup[16] + 3.812 * lineup[17] + 3.737 * lineup[18] + 3.721 * lineup[19] + 3.654 * lineup[20] + 3.643 * lineup[21] + 3.536 * lineup[22] + 3.439 * lineup[23] + 3.428 * lineup[24] + 3.401 * lineup[25] + 3.371 * lineup[26] + 3.183 * lineup[27] + 3.097 * lineup[28] + 3.095 * lineup[29] + 2.488 * lineup[30] + 1.716 * lineup[31] + 1.446 * lineup[32] + 1.384 * lineup[33] + 1.22 * lineup[34] + 0.664 * lineup[35] + 0.08 * lineup[36] + 0.4152 * bench[0] + 0.403 * bench[11] + 0.4229 * bench[15] + 0.3826 * bench[16] + 0.3812 * bench[17] + 0.3737 * bench[18] + 0.3721 * bench[19] + 0.3654 * bench[20] + 0.3643 * bench[21] + 0.3536 * bench[22] + 0.3439 * bench[23] + 0.3428 * bench[24] + 0.3401 * bench[25] + 0.3371 * bench[26] + 0.3183 * bench[27] + 0.3097 * bench[28] + 0.3095 * bench[29] + 0.2488 * bench[30] + 0.1716 * bench[31] + 0.1446 * bench[32] + 0.1384 * bench[33] + 0.122 * bench[34] + 0.0664 * bench[

In [15]:
model.add_constraints((lineup[p] + bench[p] <=1 for p in players), name='linrup_or_bench')

sasoptpy.ConstraintGroup([lineup[0] + bench[0] <=  1, lineup[11] + bench[11] <=  1, lineup[15] + bench[15] <=  1, lineup[16] + bench[16] <=  1, lineup[17] + bench[17] <=  1, lineup[18] + bench[18] <=  1, lineup[19] + bench[19] <=  1, lineup[20] + bench[20] <=  1, lineup[21] + bench[21] <=  1, lineup[22] + bench[22] <=  1, lineup[23] + bench[23] <=  1, lineup[24] + bench[24] <=  1, lineup[25] + bench[25] <=  1, lineup[26] + bench[26] <=  1, lineup[27] + bench[27] <=  1, lineup[28] + bench[28] <=  1, lineup[29] + bench[29] <=  1, lineup[30] + bench[30] <=  1, lineup[31] + bench[31] <=  1, lineup[32] + bench[32] <=  1, lineup[33] + bench[33] <=  1, lineup[34] + bench[34] <=  1, lineup[35] + bench[35] <=  1, lineup[36] + bench[36] <=  1, lineup[37] + bench[37] <=  1, lineup[38] + bench[38] <=  1, lineup[39] + bench[39] <=  1, lineup[40] + bench[40] <=  1, lineup[41] + bench[41] <=  1, lineup[42] + bench[42] <=  1, lineup[43] + bench[43] <=  1, lineup[44] + bench[44] <=  1, lineup[45] + ben

In [16]:
model.add_constraint(so.expr_sum(lineup[p] for p in players) == 1, name='single_lineup')

sasoptpy.Constraint(lineup[0] + lineup[11] + lineup[15] + lineup[16] + lineup[17] + lineup[18] + lineup[19] + lineup[20] + lineup[21] + lineup[22] + lineup[23] + lineup[24] + lineup[25] + lineup[26] + lineup[27] + lineup[28] + lineup[29] + lineup[30] + lineup[31] + lineup[32] + lineup[33] + lineup[34] + lineup[35] + lineup[36] + lineup[37] + lineup[38] + lineup[39] + lineup[40] + lineup[41] + lineup[42] + lineup[43] + lineup[44] + lineup[45] + lineup[46] + lineup[47] + lineup[48] + lineup[49] + lineup[50] + lineup[51] + lineup[52] + lineup[53] + lineup[54] + lineup[55] + lineup[56] + lineup[57] + lineup[58] + lineup[59] + lineup[60] + lineup[61] + lineup[62] + lineup[63] + lineup[64] + lineup[65] + lineup[66] + lineup[67] + lineup[68] + lineup[69] + lineup[70] + lineup[71] + lineup[72] + lineup[73] + lineup[74] + lineup[75] + lineup[76] + lineup[77] + lineup[78] + lineup[79] + lineup[80] + lineup[81] + lineup[82] + lineup[83] + lineup[84] + lineup[85] + lineup[86] ==  1, name='single_l

In [17]:
model.add_constraint(so.expr_sum(bench[p] for p in players) == 1, name='single_bench')

sasoptpy.Constraint(bench[0] + bench[11] + bench[15] + bench[16] + bench[17] + bench[18] + bench[19] + bench[20] + bench[21] + bench[22] + bench[23] + bench[24] + bench[25] + bench[26] + bench[27] + bench[28] + bench[29] + bench[30] + bench[31] + bench[32] + bench[33] + bench[34] + bench[35] + bench[36] + bench[37] + bench[38] + bench[39] + bench[40] + bench[41] + bench[42] + bench[43] + bench[44] + bench[45] + bench[46] + bench[47] + bench[48] + bench[49] + bench[50] + bench[51] + bench[52] + bench[53] + bench[54] + bench[55] + bench[56] + bench[57] + bench[58] + bench[59] + bench[60] + bench[61] + bench[62] + bench[63] + bench[64] + bench[65] + bench[66] + bench[67] + bench[68] + bench[69] + bench[70] + bench[71] + bench[72] + bench[73] + bench[74] + bench[75] + bench[76] + bench[77] + bench[78] + bench[79] + bench[80] + bench[81] + bench[82] + bench[83] + bench[84] + bench[85] + bench[86] ==  1, name='single_bench')

In [18]:
budget = 9.5
model.add_constraint(so.expr_sum((lineup[p]+ bench[p])*gk_data.loc[p,'BV'] for p in players) <=budget, name='budget_con')

sasoptpy.Constraint(5.6 * lineup[0] + 5.6 * bench[0] + 4.4 * lineup[11] + 4.4 * bench[11] + 5.1 * lineup[15] + 5.1 * bench[15] + 5.3 * lineup[16] + 5.3 * bench[16] + 5.5 * lineup[17] + 5.5 * bench[17] + 5.1 * lineup[18] + 5.1 * bench[18] + 4.7 * lineup[19] + 4.7 * bench[19] + 4.6 * lineup[20] + 4.6 * bench[20] + 5.5 * lineup[21] + 5.5 * bench[21] + 6.1 * lineup[22] + 6.1 * bench[22] + 4.5 * lineup[23] + 4.5 * bench[23] + 5.4 * lineup[24] + 5.4 * bench[24] + 5.2 * lineup[25] + 5.2 * bench[25] + 5.0 * lineup[26] + 5.0 * bench[26] + 4.8 * lineup[27] + 4.8 * bench[27] + 4.5 * lineup[28] + 4.5 * bench[28] + 4.6 * lineup[29] + 4.6 * bench[29] + 5.9 * lineup[30] + 5.9 * bench[30] + 4.5 * lineup[31] + 4.5 * bench[31] + 4.8 * lineup[32] + 4.8 * bench[32] + 4.4 * lineup[33] + 4.4 * bench[33] + 4.2 * lineup[34] + 4.2 * bench[34] + 4.5 * lineup[35] + 4.5 * bench[35] + 4.8 * lineup[36] + 4.8 * bench[36] + 5.0 * lineup[37] + 5.0 * bench[37] + 4.5 * lineup[38] + 4.5 * bench[38] + 3.9 * lineup[39] + 3

In [19]:
model.set_objective(-total_xp, name='total_xp_obj', sense='N')

sasoptpy.Expression(exp = - 4.152 * lineup[0] - 4.03 * lineup[11] - 4.229 * lineup[15] - 3.826 * lineup[16] - 3.812 * lineup[17] - 3.737 * lineup[18] - 3.721 * lineup[19] - 3.654 * lineup[20] - 3.643 * lineup[21] - 3.536 * lineup[22] - 3.439 * lineup[23] - 3.428 * lineup[24] - 3.401 * lineup[25] - 3.371 * lineup[26] - 3.183 * lineup[27] - 3.097 * lineup[28] - 3.095 * lineup[29] - 2.488 * lineup[30] - 1.716 * lineup[31] - 1.446 * lineup[32] - 1.384 * lineup[33] - 1.22 * lineup[34] - 0.664 * lineup[35] - 0.08 * lineup[36] - 0.4152 * bench[0] - 0.403 * bench[11] - 0.4229 * bench[15] - 0.3826 * bench[16] - 0.3812 * bench[17] - 0.3737 * bench[18] - 0.3721 * bench[19] - 0.3654 * bench[20] - 0.3643 * bench[21] - 0.3536 * bench[22] - 0.3439 * bench[23] - 0.3428 * bench[24] - 0.3401 * bench[25] - 0.3371 * bench[26] - 0.3183 * bench[27] - 0.3097 * bench[28] - 0.3095 * bench[29] - 0.2488 * bench[30] - 0.1716 * bench[31] - 0.1446 * bench[32] - 0.1384 * bench[33] - 0.122 * bench[34] - 0.0664 * benc

In [20]:
model.export_mps(filename='gk.mps')

P:\AnacondaN\Lib\site-packages\sasoptpy\interface\format\mps_format.py:188: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[f] = df[f].replace('', np.nan)


In [21]:
command = 'cbc gk.mps solve solu solution.txt'

In [22]:
!{command}

Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Aug 20 2024 

command line - cbc gk.mps solve solu solution.txt (default strategy 1)
At line 1 NAME    gk_model
At line 2 ROWS
At line 81 COLUMNS
At line 378 RHS
At line 418 RANGES
At line 419 BOUNDS
At line 568 ENDATA
Problem gk_model has 77 rows, 148 columns and 444 elements
Coin0008I gk_model read with 0 errors
Continuous objective value is -4.632 - 0.00 seconds
Cgl0005I 2 SOS with 148 members
Cgl0004I processed model has 77 rows, 148 columns (148 integer (148 of which binary)) and 444 elements
Cutoff increment increased from 1e-05 to 9.99e-05
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -4.632
Cbc0038I Before mini branch and bound, 148 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improve solution (0.02 seconds)
Cbc0038I After 0.02 seconds - Feasibility pump exiting with objective of -4.632 - took 0.01 seconds
Cbc0012I Integer solution of -4.632 foun

In [23]:
with open('solution.txt', 'r') as f:
    for v in model.get_variables():
        v.set_value(0)
    for line in f:
        if 'objective value' in line:
            continue
        words = line.split()
        var = model.get_variable(words[1])
        var.set_value(float(words[2]))

In [25]:
print("LINEUP")
for p in players:
    if lineup[p].get_value() > 0.5:
        print(p, gk_data.loc[p])
print("BENCH")
for p in players:
    if bench[p].get_value() > 0.5:
        print(p, gk_data.loc[p])

LINEUP
15 Pos_Num           1
Pos               G
Name          Mendy
BV              5.1
SV              5.1
Team        Chelsea
23_xMins         88
23_Pts        4.229
Name: 15, dtype: object
BENCH
11 Pos_Num            1
Pos                G
Name         Sánchez
BV               4.4
SV               4.4
Team        Brighton
23_xMins          88
23_Pts          4.03
Name: 11, dtype: object
